# Notes about SNAP
1. snap.GetBetweennessCentr(PNGraph, Nodes, Edges, InDist=False) equals to snap.GetBetweennessCentr(PUNGraph, Nodes, Edges)
2. snap.GetDegreeCentr(UG, nid) == float(n.GetDeg())/(size-1)
3. http://snap.stanford.edu/class/cs224w-2013/recitation/snappy/Snappy-tutorial-2013.pdf

# Notes about func intxn_from_segs(segs, directionality_column=None):
    
    # check whether the intersection output is correct, by picking some examples for both B, FT, TF. 
    1. FT/0 type: DC STREETSEGID = 876(-77.01239193659748 38.950836102255), checked; 14713(-76.9902413493931 38.8773984704994), checked; 8891(-77.01366204332678 38.899295772364), checked
    2. TF/1 type: DC STREETSEGID = 13161(-77.01163764014346 38.953383268855), checked; 8292(-77.0100561260908 38.9555132405171), checked; 92(-77.01618238255742 38.896139800171), checked
    3. B/2 type: DC STREETSEGID = 10395(-77.0116155077942 38.9556194835925), checked; 10090(-77.0116172263626 38.9576690437764), checked; 15306(-77.01885842791256 38.836994168269), checked; 15284(-76.99377942384207 38.875606235068)



1. feature for segs as node:
    - N
        - [x] in/out degree
        - [x] betweenness centrality(https://snap.stanford.edu/snappy/doc/reference/GetBetweennessCentr.html)
        - [x] closeness centrality(https://snap.stanford.edu/snappy/doc/reference/GetClosenessCentr.html)
        - [x] Farness centrality(https://snap.stanford.edu/snappy/doc/reference/GetFarnessCentr.html)
        - [x] page rank(https://snap.stanford.edu/snappy/doc/reference/GetPageRank.html)
        - [x] Hits score(https://snap.stanford.edu/snappy/doc/reference/GetHits.html)
        - [x] node eccentricity(https://snap.stanford.edu/snappy/doc/reference/GetNodeEcc.html)
    - UN
        - [x] degree centrality(https://snap.stanford.edu/snappy/doc/reference/GetDegreeCentr.html)
        - [x] eigenvector centrality(https://snap.stanford.edu/snappy/doc/reference/GetEigenVectorCentr.html)
        - [x] betweenness centrality(https://snap.stanford.edu/snappy/doc/reference/GetBetweennessCentr.html)
        - [x] closeness centrality(https://snap.stanford.edu/snappy/doc/reference/GetClosenessCentr.html)
        - [x] Farness centrality(https://snap.stanford.edu/snappy/doc/reference/GetFarnessCentr.html)
        - [x] page rank(https://snap.stanford.edu/snappy/doc/reference/GetPageRank.html)
        - [x] Hits score(https://snap.stanford.edu/snappy/doc/reference/GetHits.html)
        - [x] node eccentricity(https://snap.stanford.edu/snappy/doc/reference/GetNodeEcc.html)
        - [x] articulation pointshttps://snap.stanford.edu/snappy/doc/reference/GetArtPoints.html
        - [x] bridge(https://snap.stanford.edu/snappy/doc/reference/GetEdgeBridges.html)

In [1]:
import snap
import geopandas as gp
import pandas as pd

In [3]:
import src.constants;reload(src.constants)
import src.geom_helper; reload(src.geom_helper)
from src.geom_helper import intxn_from_segs
from src.constants import fn_intxn_dc, fn_segments_dc, index_seg, fn_feature_seg_as_node_dc
from src.constants import fn_segments_ph, epsg_ph, latlon_crs, fn_feature_seg_as_node_ph

In [4]:
import src.ftr_network; reload(src.ftr_network)
from src.ftr_network import *

In [5]:
data_dir = 'data/'

In [7]:
# path_segs_ph = data_dir + fn_segments_ph
# intxn_directed = intxn_from_segs(path_segs_ph, directionality_column='ONEWAY')
# intxn_directed.shape

In [47]:
# from shapely.ops import linemerge
# path_segs_dc = data_dir + fn_segments_dc
# segs_dc = gp.read_file(path_segs_dc)

# segs_dc.geometry = segs_dc.geometry.apply(lambda x: x if x.type!='MultiLineString' else linemerge(x))

# intxn_directed_dc = intxn_from_segs(segs_dc, directionality_column='DIRECTIONALITY', debug=True)
# intxn_directed_dc.shape

intersction project {(0.0, 1.0): 43, (1.0, 0.0): 44, (0.0,): 33141, (): 488, (1.0,): 32690}


(65918, 14)

In [8]:
path_segs_ph = data_dir + fn_segments_ph
path_segs_dc = data_dir + fn_segments_dc
path_ftr_SgAsNd_dc = data_dir + fn_feature_seg_as_node_dc
path_ftr_SgAsNd_ph = data_dir + fn_feature_seg_as_node_ph

In [ ]:
ftr_network_SgAsNd(path_segs_dc, path_ftr_SgAsNd_dc, 'DIRECTIONALITY', True)

In [ ]:
ftr_network_SgAsNd(path_segs_ph, path_ftr_SgAsNd_ph, 'ONEWAY', True)


def ftr_segs_as_nodes(nodes, edges, is_dird=False):

    # TODO: remove arg:is_dird with directionality_column
    # TODO: modify the feature name and function name, add SgAsNd
    # TODO: reconsider the function name, because processs of building network of SgAsNd and SgAsEg is quite different


In [ ]:
    path_segs= path_segs_dc
    directionality_column = 'DIRECTIONALITY'

In [124]:
def ftr_netw_SgAsEg(path_segs, path_ftr_seg_as_edge, directionality_column=None):
    from collections import defaultdict
    segs = gp.read_file(path_segs)
    segs.index.name = index_seg
    # get from/to point of each segment
    segs['f'] = segs.geometry.apply(lambda x: x.coords[0])
    segs['t'] = segs.geometry.apply(lambda x: x.coords[-1])
    # index nodes
    node2idx = {n:i for i,n in enumerate(pd.unique(segs[['f','t']].values.flatten()))}
    idx2node = {i:n for n,i in node2idx.items()}
    nodes = idx2node.keys()
    # indexize nodes in segments
    segs['f'] = segs.f.apply(lambda x: node2idx[x])
    segs['t'] = segs.t.apply(lambda x: node2idx[x])

    edge2idx = defaultdict(list)

    for sidx, drtn, f, t in segs[[directionality_column, 'f', 't']].reset_index().values:
        if f == t:
            edge2idx[(f,t)].append(sidx)
            continue
        if drtn=='B' or drtn==2:
            edge2idx[(f,t)].append(sidx)
            edge2idx[(t,f)].append(sidx)
        elif drtn=='FT' or drtn==0:
            edge2idx[(f,t)].append(sidx)
        elif drtn=='TF' or drtn==1:
            edge2idx[(t,f)].append(sidx)

    edges = edge2idx.keys()
    print '# nodes = {}, # edges = {}'.format(len(nodes), len(edges))

    G = snap.TNGraph.New()
    for n in nodes:
        G.AddNode(int(n))
    for ein, eout in edges:
        G.AddEdge(int(ein), int(eout))
    UG = snap.ConvertGraph(snap.PUNGraph, G)
    print 'node size = {}, directed edges = {}, undirected edges = {}'.format(len(nodes), len([0 for e in G.Edges()]), len([0 for e in UG.Edges()]))
    return edge2idx, G, UG

In [125]:
edge2idx_dc, G_dc, UG_dc = ftr_netw_SgAsEg(path_segs_dc, '', directionality_column='DIRECTIONALITY')

# nodes = 8520, # edges = 24846
node size = 8520, directed edges = 24846, undirected edges = 13478


In [105]:
def get_segidxs_in_ug(f, t):
    segidxs1 = edge2idx[(f,t)]
    segidxs2 = edge2idx[(t,f)]
    return list(set(segidxs1+segidxs2))
    

In [106]:
features = defaultdict(lambda: defaultdict(int))

sidx_history_btw_d = []
Nodesbd = snap.TIntFltH()
Edgesbd = snap.TIntPrFltH()
snap.GetBetweennessCentr(G, Nodesbd, Edgesbd, 1.0, True)
for edge in Edgesbd:
    segidxs = edge2idx[(edge.GetVal1(), edge.GetVal2())]
    for sidx in segidxs:
        sidx_history_btw_d.append(sidx)
        features[sidx]['d_btw_cntr_SgAsEd'] += Edgesbd[edge]

sidx_history_btw_ud = []
Nodesbud = snap.TIntFltH()
Edgesbud = snap.TIntPrFltH()
snap.GetBetweennessCentr(UG, Nodesbud, Edgesbud, 1.0)
for edge in Edgesbud:
    segidxs = get_segidxs_in_ug(edge.GetVal1(), edge.GetVal2())
    for sidx in segidxs:
        sidx_history_btw_ud.append(sidx)
        features[sidx]['ud_btw_cntr_SgAsEd'] += Edgesbud[edge]
        
sidx_history_bridge_ud = []
EdgeV = snap.TIntPrV()
snap.GetEdgeBridges(UG, EdgeV)
for edge in EdgeV:
    segidxs = get_segidxs_in_ug(edge.GetVal1(), edge.GetVal2())
    for sidx in segidxs:
        sidx_history_bridge_ud.append(sidx)
        features[sidx]['ud_bridge_SgAsEd'] += 1

In [107]:
df = pd.DataFrame.from_dict(features).T


# TODO
1. [x] get segid by edge: Directed Graph --> Undirected Graph: 
    - src and dir of an edge may change: (107,0) --> (0, 107); (8407, 8406) --> (8406, 8407) [seems that always smaller nodeid points to larger nodeid]
2. check whether the features is correctly assigned to segments, check cases:
    - one directed edges --> one segid: E.g. index_seg=[13482,2438,2,1,86,0]
    - two directed edges --> one segid: E.g. index_seg=[2818,2819,2773,5788,2772,5787,2243,2242,3,13436,13466,13474]
    - one directed edge --> two segids: E.g. index_seg=[[10247, 10568],[10368, 10570],[13093, 13278]]
    - two directed edges --> two segids: E.g. index_seg=[[952, 2227], [2234, 2238],[12597, 13219]]
    - loop edges --> one segid: E.g. index_seg=[12412, 13055, 11916]
        - betweenness centerality: they don't have value in undirect graph; value=0 in directed graph

In [136]:
df = pd.DataFrame(edge2idx_dc.items(), columns=['e','sidx'])
df['len_sidx'] = df.sidx.apply(len)
df['ud'] = df.e.apply(lambda x: tuple(sorted(x)))
df['loop'] = df.e.apply(lambda x: x[0]==x[1])

In [139]:
df[df['loop']]

,e,sidx,len_sidx,ud,loop
3297,"(8039, 8039)",[12412],1,"(8039, 8039)",True
3733,"(8353, 8353)",[13055],1,"(8353, 8353)",True
6851,"(8336, 8336)",[12993],1,"(8336, 8336)",True
7619,"(6606, 6606)",[10740],1,"(6606, 6606)",True
8512,"(5060, 5060)",[7004],1,"(5060, 5060)",True
12585,"(8316, 8316)",[12888],1,"(8316, 8316)",True
14168,"(3330, 3330)",[3882],1,"(3330, 3330)",True
14353,"(3139, 3139)",[13336],1,"(3139, 3139)",True
14997,"(3873, 3873)",[5741],1,"(3873, 3873)",True
15345,"(8295, 8295)",[12837],1,"(8295, 8295)",True


In [137]:
df.sort('ud')

,e,sidx,len_sidx,ud,loop
22405,"(0, 1)",[0],1,"(0, 1)",False
13239,"(25, 0)",[3267],1,"(0, 25)",False
24218,"(0, 25)",[3267],1,"(0, 25)",False
17852,"(107, 0)",[86],1,"(0, 107)",False
10073,"(2782, 1)",[2818],1,"(1, 2782)",False
13395,"(1, 2782)",[2818],1,"(1, 2782)",False
24268,"(2783, 1)",[2819],1,"(1, 2783)",False
8235,"(1, 2783)",[2819],1,"(1, 2783)",False
15574,"(3, 2)",[1],1,"(2, 3)",False
15890,"(2, 27)",[2773],1,"(2, 27)",False


In [138]:
df[df.len_sidx==2].sort('ud')

,e,sidx,len_sidx,ud,loop
6092,"(1068, 1069)","[952, 2227]",2,"(1068, 1069)",False
15446,"(1069, 1068)","[952, 2227]",2,"(1068, 1069)",False
6552,"(2458, 2459)","[2234, 2238]",2,"(2458, 2459)",False
22764,"(2459, 2458)","[2234, 2238]",2,"(2458, 2459)",False
19886,"(2519, 2520)","[2322, 5321]",2,"(2519, 2520)",False
5356,"(2520, 2519)","[2322, 5321]",2,"(2519, 2520)",False
13193,"(2925, 2924)","[3080, 4331]",2,"(2924, 2925)",False
1882,"(2924, 2925)","[3080, 4331]",2,"(2924, 2925)",False
5777,"(3052, 3066)","[3332, 3574]",2,"(3052, 3066)",False
7154,"(3066, 3052)","[3332, 3574]",2,"(3052, 3066)",False


In [54]:
len(list(UG.Edges())), len(Edgesbd), len(Edgesbud), len(list(UG.Edges()))- len(Edgesbud)

(13478, 24846, 13459, 19)

In [51]:
ug_edges = [(e.GetSrcNId(), e.GetDstNId()) for e in UG.Edges()]
bud_edges = [(edge.GetVal1(), edge.GetVal2()) for edge in Edgesbud]

In [56]:
g_edges = [(e.GetSrcNId(), e.GetDstNId()) for e in G.Edges()]
bd_edges = [(edge.GetVal1(), edge.GetVal2()) for edge in Edgesbd]

In [58]:
edge2idx[(2942, 2942)]

[3111]

In [67]:
self_edges_idxs = [edge2idx[e][0] for e in list(set(ug_edges) - set(bud_edges))]

In [29]:
edge2idx[(107,0)]

[86]

In [72]:
%matplotlib inline

In [102]:
cnt_opp = 0
cnt_not_exist = 0
cnt_not_equal = 0
# cnt_opp, cnt_not_exist, cnt_not_equal
for edge in Edgesbud:
#     segidxs1 = edge2idx[(edge.GetVal1(), edge.GetVal2())]
#     segidxs2 = edge2idx[(edge.GetVal2(), edge.GetVal1())]    
#     if segidxs1 and segidxs2:
#         if segidxs1!=segidxs2:
#             cnt_not_equal += 1
#             a= 
#             print 'not equal', (edge.GetVal1(), edge.GetVal2()), segidxs1, segidxs2, a
#     elif not (segidxs1 or segidxs2):
#         cnt_not_exist += 1
#         print 'not exist', (edge.GetVal1(), edge.GetVal2()), segidxs
#     elif not segidxs1:
#         cnt_opp+=1
#         print 'opp direc', (edge.GetVal1(), edge.GetVal2()), segidxs, segidxs2, edge.GetVal1() < edge.GetVal2()
    segidxs = get_segidxs_in_ug(edge.GetVal1(), edge.GetVal2())
    for sidx in segidxs:
        features[sidx]['ud_btw_cntr_SgAsEd'] += Edgesbud[edge]


# deprecated
building network using intersection file from open dc. Because the intxn file isn't complete

In [32]:
path_intxn_dc = data_dir+fn_intxn_dc
path_segs_dc = data_dir + fn_segments_dc
path_ftr_seg_as_node_dc = data_dir + fn_feature_seg_as_node

In [5]:
df_dc_seg_as_node = ftr_for_dc(path_intxn_dc, path_segs_dc, path_ftr_seg_as_node_dc)

node size = 13443, directed edges = 18006, undirected edges = 14147
